### Import Modules

In [49]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time
from importlib import reload

sys.path.append("../")
from fitter import data_loader as dl
from fitter import bootstrapper as bs
from fitter import fitter as fit
from fitter import special_functions as sf

for j in range(10): # Sometimes this needs to be loaded twice...
    matplotlib.rcParams['figure.figsize'] = [10, 10]

In [50]:
reload(dl)
reload(sf)
reload(fit)
reload(bs)

<module 'fitter.bootstrapper' from '../fitter/bootstrapper.py'>

### Specify fit parameters

In [51]:
p_dict = {
    'order' : {
        'fit' : 'nnlo', # 'nlo', 'nnlo'
        'vol' : 0, # max 10
        'exclude' : [], # put LECs here
        
        # semi-nnlo corrections
        'include_alpha_s' : True,
        'include_log' : True, 
        'include_log2' : True,
        'include_sunset' : True,
        
        # nnnlo corrections
        'include_latt_n3lo' : True,
    }, 
    'fit_type' : 'xpt-ratio', #'ma', 'ma-ratio, 'xpt', or 'xpt-ratio'; experimental: 'simultaneous', 'ma-old'
    'F2' : 'FKFpi', #'FKFpi', 'FpiFpi', 'FKFK', 'F0F0' (HPQCD choice)
    'use_bijnens_central_value' : False,
    
    'use_prior' : True,
    'bias_correct' : True,
    'include_su2_isospin_corrrection' : False,
    'abbrs' : [u'a06m310L', u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m220L', u'a12m310', u'a12m350',
           u'a12m400',  u'a15m135XL', u'a15m220', u'a15m310', u'a15m350', 'a15m400'], # u'a15m130'
    
    'make_plots' : False,
    'show_error_ellipses' : False,
    'show_bs_histograms' : False,
    
    'save_results' : False,
    'save_pickles' : False,
    'output_name' : 'fit_results',
}

In [52]:
t0 = time.time()

# Load data
data_loader = dl.data_loader()
fit_data = data_loader.get_fit_data()
phys_point_data = data_loader.get_phys_point_data()

# Get prior
prior = None
if p_dict['use_prior']:
    prior = data_loader.get_prior(fit_type=p_dict['fit_type'], order=p_dict['order']['fit'], F2=p_dict['F2'], 
                  include_log=p_dict['order']['include_log'], include_log2=p_dict['order']['include_log2'], 
                  include_sunset=p_dict['order']['include_sunset'], include_alpha_s=p_dict['order']['include_alpha_s'], 
                  include_latt_n3lo=p_dict['order']['include_latt_n3lo'], include_FV=(p_dict['order']['vol'] > 6), 
                  use_bijnens_central_value=p_dict['use_bijnens_central_value']
             )
    
# Make bootstrapper
bootstrapper = bs.bootstrapper(fit_data, phys_point_data, prior=prior, **p_dict)

if p_dict['make_plots']:
    data_loader.save_plots(
        bootstrapper.make_plots(
            show_error_ellipses=p_dict['show_error_ellipses'], 
            show_bootstrap_histograms=p_dict['show_bs_histograms']),
        output_filename='fits/'+bootstrapper.fit_type+'_'+bootstrapper.order['fit']
    )
else:
    print(bootstrapper)
    
if p_dict['save_results']:
    data_loader.save_fit_info(bootstrapper.get_fit_info(),
                              output_name=p_dict['output_name'], save_pickles=p_dict['save_pickles'])

    
    
t1 = time.time()

print("\nTotal time (s): ", t1 - t0, "\n")

/mnt/Google/Documents/School Work/Research/fk-fpi/priors/xpt-ratio.csv
Making fits...
Done!
100% complete
Time (s):  8.716484308242798
Compiling results...

Model: xpt-ratio_FKFpi_nnlo_log_logSq_sunset_alphaS_a4

Fitting to nnlo 
 with volume corrections O(0) 
Fitted/[FLAG] values at physical point (including SU(2) isospin corrections: False):

F_K / F_pi = 1.1899(51) [1.1956(19)]   (delta_su2 = -0.00408(70))

  Parameter    Result[0] / Prior[1]    Parameter    Result[0] / Prior[1]
-----------------------------------  -----------------------------------
      A_a 0              -6.3 (3.5)            1            0.00000 (22)
          1               0.0 (6.5)        L_3 0            -0.0005 (28)
     A_aa 0                 56 (22)            1             0.0000 (31)
          1                  0 (54)        L_4 0           -0.00008 (34)
      A_k 0               2.4 (2.5)            1             0.0000 (50)
          1               0.0 (5.8)        L_5 0             0.0006 (11)
  

In [84]:
m, p = gv.gvar([5, 4], [[0.1, 0.01], [0.02, 0.2]])
E = np.sqrt(m**2 + p**2)

print('E =', E, '\n')
print(gv.fmt_errorbudget(outputs={'E' : E}, inputs={'m' : m, 'p' : p}, percent=False, verify=True))

E = 6.40(39) 

Partial Errors:
                   E
--------------------
        m:      0.39
        p:      0.39
--------------------
    total:      0.39



<ipython-input-84-52bc4adc7b08>:5: UserWarning: m and p double count errors
  print(gv.fmt_errorbudget(outputs={'E' : E}, inputs={'m' : m, 'p' : p}, percent=False, verify=True))
<ipython-input-84-52bc4adc7b08>:5: UserWarning: E partial error 0.5543618612158775  !=  total error 0.3919930312969427
  print(gv.fmt_errorbudget(outputs={'E' : E}, inputs={'m' : m, 'p' : p}, percent=False, verify=True))


In [75]:
E

6.40(39)

In [140]:
    data_loader.save_fit_info(bootstrapper.get_fit_info(),
                              output_name='test', save_pickles=True)

Saving...
a/w0
a
L
alpha_s
mpi
mk
mss
a2DI
Fpi
FK
w0
FK/Fpi_pm
mju
mru
mjs
mrs
lam2_chi_kpi
lam2_chi_pipi
lam2_chi_kk
lam2_chi_00
lam2_chi
Done.


In [26]:
fit_info = data_loader.get_fit_info('test')['xpt-ratio_FKFpi_nnlo_log_logSq_sunset_alphaS_a4']
fit_info['FK/Fpi'].partialsdev(fit_info['phys_point']['lam2_chi'])

NameError: name 'data_loader' is not defined

In [96]:
def test(self):
    p = self.get_fit_info()['posterior']
    
    print(gv.evalcorr([p['A_a'], p['A_k'], p['A_p'], p['L_4'], p['L_5']]))
    
test(bootstrapper)

[[ 1.          0.14852927  0.16854288  0.15795649 -0.14118969]
 [ 0.14852927  1.          0.78017949  0.40069967 -0.32779122]
 [ 0.16854288  0.78017949  1.          0.09405687  0.00951327]
 [ 0.15795649  0.40069967  0.09405687  1.         -0.99295323]
 [-0.14118969 -0.32779122  0.00951327 -0.99295323  1.        ]]


In [91]:
print(gv.evalcorr(bootstrapper.get_fit_info()['posterior']))

{('A_a', 'A_a'): array([[1.]]),('A_a', 'A_aa'): array([[-0.17241389]]),('A_a', 'A_k'): array([[0.14852927]]),('A_a', 'A_loga'): array([[-0.88879273]]),('A_a', 'A_p'): array([[0.16854288]]),('A_a', 'L_1'): array([[-0.00492822]]),('A_a', 'L_2'): array([[-0.00841322]]),('A_a', 'L_3'): array([[-0.06826485]]),('A_a', 'L_4'): array([[0.15795649]]),('A_a', 'L_5'): array([[-0.14118969]]),('A_a', 'L_6'): array([[-0.00303813]]),('A_a', 'L_7'): array([[-0.02727483]]),('A_a', 'L_8'): array([[-0.00830673]]),('A_aa', 'A_a'): array([[-0.17241389]]),('A_aa', 'A_aa'): array([[1.]]),('A_aa', 'A_k'): array([[-0.49685592]]),('A_aa', 'A_loga'): array([[-0.28948502]]),('A_aa', 'A_p'): array([[-0.37722297]]),('A_aa', 'L_1'): array([[0.01402727]]),('A_aa', 'L_2'): array([[0.02398072]]),('A_aa', 'L_3'): array([[0.19645954]]),('A_aa', 'L_4'): array([[-0.414266]]),('A_aa', 'L_5'): array([[0.37461416]]),('A_aa', 'L_6'): array([[0.00915482]]),('A_aa', 'L_7'): array([[0.07557115]]),('A_aa', 'L_8'): array([[0.023192

In [ ]:
aa

In [98]:
def error_budget(self):
    fit_info = self.get_fit_info()
    
    inputs = {}
    for key in list(fit_info['prior']):
        inputs['prior:'+key] = fit_info['prior']
    for key in list(fit_info['posterior']):
        inputs['posterior:'+key] = fit_info['posterior']
    for key in list(fit_info['phys_point']):
        inputs['phys_point:'+key] = fit_info['phys_point']
        
    print('FK/Fpi =', fit_info['FK/Fpi'], '\n')
        
    return gv.fmt_errorbudget(outputs={'FK/Fpi' : fit_info['FK/Fpi']}, inputs=inputs, percent=False, ndecimal=5, verify=False)

print(error_budget(bootstrapper))

FK/Fpi = 1.1899(51) 

Partial Errors:
                             FK/Fpi
-----------------------------------
               prior:A_a:   0.00355
              prior:A_aa:   0.00355
               prior:A_k:   0.00355
            prior:A_loga:   0.00355
               prior:A_p:   0.00355
               prior:L_1:   0.00355
               prior:L_2:   0.00355
               prior:L_3:   0.00355
               prior:L_4:   0.00355
               prior:L_5:   0.00355
               prior:L_6:   0.00355
               prior:L_7:   0.00355
               prior:L_8:   0.00355
           posterior:A_a:   0.00502
          posterior:A_aa:   0.00502
           posterior:A_k:   0.00502
        posterior:A_loga:   0.00502
           posterior:A_p:   0.00502
           posterior:L_1:   0.00502
           posterior:L_2:   0.00502
           posterior:L_3:   0.00502
           posterior:L_4:   0.00502
           posterior:L_5:   0.00502
           posterior:L_6:   0.00502
           posterior:L_7: 

In [34]:
fit_data = {
    'mpi' : 135.,
    'mk' : 495.5,
    'Fpi' : 92.2,
    'FK' : 110.5,
    'L' : np.inf, 
    'a/w0' : 0,
}
fit_data['lam2_chi'] = (4 *np.pi *fit_data['Fpi'])**2

fit_parameters = {
    'L_1' : 0.000372,
    'L_2' : 0.000493,
    'L_3' : -0.003070,
    'L_4' : 0.000089,
    'L_5' : 0.000377,
    'L_6' : 0.000011,
    'L_7' : -0.000340,
    'L_8' : 0.000294,
    'A_a' : 0.1,
    'A_k' : -3.000000,
    'A_p' : 4.000000
}


bootstrapper.fk_fpi_fit_fcn(fit_data=fit_data, fit_parameters=fit_parameters, debug=True)

NameError: name 'bootstrapper' is not defined